In [ ]:
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from tensorflow.keras.models import load_model
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

In [ ]:
data = pd.read_csv('/Users/veda/Downloads/Bank_Personal_Loan_Modelling.csv')

In [ ]:
data.head()

In [ ]:
data.drop(["ID", "ZIP Code"],axis=1,inplace=True)

In [ ]:
x = data['Personal Loan']
data = data.drop(['Personal Loan'],axis=1)
data = data.join(x)

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
x = data.iloc[:,:-1].values
y = data.iloc[:,-1].values
enc = OneHotEncoder()
y = enc.fit_transform(y[:, np.newaxis]).toarray()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=69)
(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
def create_custom_model(nodes, n=1, name='model'):
    model = Sequential(name=name)
    for i in range(n):
        model.add(Dense(nodes, input_dim=13, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [ ]:
n_layers = 1
model = create_custom_model(4, n_layers)
model.summary()

score = model.evaluate(X_test, Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
def fitness_function(weights):
    y_pred = model(X_train)
    y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
    accuracy = (y_pred == Y_train).sum().float().item() / len(test.dataset)
    return accuracy

In [ ]:
def ant_colony_optimization(num_ants, num_iterations, alpha=1.0, beta=1.0, evaporation=0.5, Q=100, tau0=0.1):
    
    num_weights = len(model.get_weights())
    pheromone = np.full((num_weights,), tau0)

    best_solution = None
    best_fitness = float('-inf')

    for i in range(num_iterations):
        solutions = []
        fitness_values = []

        for j in range(num_ants):
            solution = np.zeros((num_weights,))

            for k in range(num_weights):
                probabilities = np.zeros((2,))
                for l in range(2):
                    delta_tau = Q / (1 + np.exp(-beta * pheromone[k]))
                    decision = model.get_weights()[k] + (l - 0.5) * alpha * delta_tau
                    model.set_weights(model.get_weights()[k] + (l - 0.5) * alpha * delta_tau)
                    probabilities[l] = fitness_function(model.get_weights())
                    model.set_weights(solution)

                decision = np.random.choice(2, p=probabilities / np.sum(probabilities))
                solution[k] = model.get_weights()[k] + (decision - 0.5) * alpha

            fitness = fitness_function(model.get_weights())

            solutions.append(solution)
            fitness_values.append(fitness)

        for k in range(num_weights):
            for l in range(2):
                delta_tau = 0
                for m in range(num_ants):
                    if solutions[m][k] == l:
                        delta_tau += fitness_values[m]
                pheromone[k] = (1 - evaporation) * pheromone[k] + delta_tau

        if np.max(fitness_values) > best_fitness:
            best_solution = solutions[np.argmax(fitness_values)]
            best_fitness = np.max(fitness_values)

    return best_solution

In [ ]:
best_weights = ant_colony_optimization(num_ants=10, num_iterations=100, alpha=1.0, beta=1.0, evaporation=0.5, Q=100, tau0=0)
loss, accuracy = model.evaluate(X_test, y_test)
print('Accuracy:', accuracy)